### **Fine Tune Embeddings Advanced RAG with Mistral AI LLM**

This notebook demonstrates how to fine-tune embeddings using BeyondLLM and build an advanced Retrieval-Augmented Generation (RAG) pipeline with Mistral AI LLM.

- **Objective:** Enhance the RAG pipeline by fine-tuning embeddings and integrating the Mistral AI LLM for improved response generation.
- **Setup:**
  - Load and preprocess a dataset for training.
  - Fine-tune the embedding model with BeyondLLM.
  - Set up a retriever and generation pipeline.
- **Steps:**
  - Train the embedding model with the provided dataset.
  - Configure the retriever with fine-tuned embeddings.
  - Initialize the Mistral AI LLM and set up the generation pipeline.
  - Generate responses to queries and assess the pipeline’s performance.
- **Outcome:** Understand the impact of fine-tuning on the RAG system's performance and how it enhances the quality of generated responses.


## Setup and Installation

Let's start by installing the necessary packages:

In [ ]:
!pip install beyondllm
!pip install llama-index-finetuning==0.1.5 llama-index-embeddings-huggingface==0.2.0

Next, we'll import the required libraries and set up the environment:

In [1]:
from beyondllm import source, retrieve, llms, generator
from beyondllm.embeddings import FineTuneEmbeddings
import os

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


## Load the data.
 Here lets use a PDF:

[The Last Question](https://docs.google.com/file/d/0ByoueGSWXluVVUtHYnRJVEg4YnM/edit?resourcekey=0-piVl8D4gdQhTyiaImqjduQ). Download it and upload on Colab or save within your project directory

In [11]:
path = "/content/TheLastQuestion.pdf"

data = source.fit(path, dtype="pdf", chunk_size=1024, chunk_overlap=0)
list_of_files = [path]

## Define Open Source LLM

In [3]:
from beyondllm.llms import HuggingFaceHubModel

from getpass import getpass

HF_TOKEN = getpass("Access token:")

Access token:··········


In [4]:
llm = HuggingFaceHubModel(
    token = HF_TOKEN,
    model = "mistralai/Mistral-7B-Instruct-v0.2",
    model_kwargs = {"max_new_tokens": 1024,
                    "temperature": 0.1,
                    "top_p": 0.95,
                    "repetition_penalty": 1.1,
                    "return_full_text": False
                  }
)

## Fine tune the Embedding model

In [5]:
embed_model = FineTuneEmbeddings()

fine_tuned_model = embed_model.train(list_of_files, "BAAI/bge-small-en-v1.5", llm, "fintune")

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Parsed 6 nodes


Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Parsed 3 nodes


100%|██████████| 3/3 [00:00<00:00, 17.38it/s]


...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2 [00:00<?, ?it/s]

## Build the retriever

In [12]:
# Option to load an already fine-tuned model
# embed_model = fine_tuned_model.load_model("fintune")

# Creating a retriever using the fine-tuned embeddings
retriever = retrieve.auto_retriever(data, fine_tuned_model, type="normal", top_k=4)

## Define generate pipeline

In [13]:
query = "what did Lupov reply to When the sun is done, the other stars will be gone, too."

In [14]:
prompt = f"<s>[INST] {query} [/INST]"

In [15]:
pipeline = generator.Generate(question=prompt,retriever=retriever,llm=llm)

In [16]:
print(pipeline.call())

 Lupov replied: "Darn right they will."

Context and chat history indicate that Lupov and Adell were discussing the fact that all stars, including the sun, will eventually run out of fuel and die. They were considering the implications of this, including the eventual end of all life in the universe due to the increase of entropy (the amount of disorder in the universe). Lupov acknowledged that other stars will also eventually die, agreeing with Adell's statement.


## Conclusion
In this notebook, we fine-tuned an embedding model using BeyondLLM, configured a retrieval pipeline, and evaluated the Mistral AI model's performance in answering a specific query. This approach allows us to improve model accuracy and tailor responses to the dataset in use.

Feel free to experiment with different parameters, datasets, and models to explore further capabilities.